<a href="https://colab.research.google.com/github/aim56009/AI-MA_project/blob/main/Masterproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import glob
import os

In [2]:
!git clone https://github.com/aim56009/AI-MA_project.git

Cloning into 'AI-MA_project'...
remote: Enumerating objects: 1818, done.
remote: Counting objects: 100% (1818/1818), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 1818 (delta 1762), reused 1780 (delta 1747), pack-reused 0
Receiving objects: 100% (1818/1818), 3.86 MiB | 8.97 MiB/s, done.
Resolving deltas: 100% (1762/1762), done.
Checking out files: 100% (1846/1846), done.


# Dataloader

transpose data in datloader!!


In [3]:
batch_size = 1      # later try diff.
PATH_TO_DATA = "AI-MA_project/pianoroll"
workers = 0

In [4]:
class MusicDataset(Dataset):

    def __init__(self, data_dir, labels, n, transforms=None):
        self.transforms = transforms
        self.data_dir = data_dir
        # load all files like below  (maybe do dicts instead of 1 big array)
        # make list of dict.  -voice_0 ..voice_all, length (-6 in total)


        for iLabel in range(len(labels)):
            file_names = sorted(glob.glob(os.path.join(data_dir, labels[iLabel], "*.pkl")))    

            if iLabel == 0:
                self.data = np.array(file_names[:n])
                self.labels = np.array(np.repeat(labels[iLabel], len(self.data)))
                
            else:
                self.data = np.append(self.data, file_names[:n])
                self.labels = np.append(self.labels, np.repeat(labels[iLabel], len(self.data)))
                
        #with open(self.data[idx]  ,'rb') as f:     ### load all data here (list,or np)   sort it after len(rows)
        #  loaded_obj = pickle.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):    ###maybe later revert sorted idx to file name idx
        print(self.data[idx])

        with open(self.data[idx]  ,'rb') as f:
          loaded_obj = pickle.load(f)
    
        pr_load = torch.tensor(loaded_obj.T)      # transpose np array first


        sample = {'pr': pr_load.float(), 'label': self.labels[idx]}     # also output the len 
        return sample

In [ ]:
# ###

In [5]:
labels = ["voice_0", "voice_1", "voice_2", "voice_3", "voice_all"]
#labels = ["voice_all","voice_0"]
dataset = MusicDataset(PATH_TO_DATA, labels, 368)

In [6]:
loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)

option: only BS=1

in dataloader:  pad_colate ----> consider only unpaded data for loss in training

 pad to longest seq in batch ..- better sort after len before loading batches

In [7]:
for batch_idx, real in enumerate(loader):
  real = real["pr"]
  print(real.shape)

AI-MA_project/pianoroll/voice_0/voice_0_001.pkl
AI-MA_project/pianoroll/voice_0/voice_0_002.pkl
AI-MA_project/pianoroll/voice_0/voice_0_003.pkl
AI-MA_project/pianoroll/voice_0/voice_0_004.pkl


RuntimeError: ignored

# Model - RNN

In [ ]:
#https://github.com/fosfrancesco/pkspell/blob/main/src/models/models.py

In [ ]:
class MusicRNN(nn.Module):

    def __init__(self, pitch_to_ix, output_dim, input_dim=17, hidden_dim=300, rnn_depth=1, cell_type="GRU"):
        super(MusicRNN, self).__init__()

        self.n_out = output_dim
        
        
        rnn_cell = nn.GRU

        self.rnn = rnn_cell(
            input_size=input_dim,
            hidden_size= hidden_dim,
            num_layers=rnn_depth)


        # Output layers. The input will be two times
        self.top_layer_voice_0 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_1 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_2 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_3 = nn.Linear(hidden_dim, self.n_out)

        # Loss function that we will use during training.
        self.loss = nn.CrossEntropyLoss(reduction="mean")


    def compute_outputs(self, sentences, sentences_len):

        # pack padded seq
        rnn_out = self.rnn(sentences)
        # pad packed seq

        out_0 = self.top_layer_voice_0(rnn_out)
        out_1 = self.top_layer_voice_1(rnn_out)
        out_2 = self.top_layer_voice_2(rnn_out)
        out_3 = self.top_layer_voice_3(rnn_out)
        

        return out_0, out_1, out_2, out_3

    def forward(self, sentences, v0, v1, v2, v3, sentences_len):

        # Compute the outputs. The shape is (max_len, n_sentences, n_labels).
        scores_0, scores_1, scores_2, scores_3 = self.compute_outputs(sentences, sentences_len)

        # Flatten the outputs and the labels, to compute the loss.
        # The input to this loss needs to be one 2-dimensional and one 1-dimensional tensor.
        score_0  = scores_0.view(-1, self.n_out)
        score_1  = scores_1.view(-1, self.n_out)
        score_2  = scores_2.view(-1, self.n_out)
        score_3  = scores_3.view(-1, self.n_out)

        # dont take padded len -> loss(score_0[:sentence_len], v0)      -> therefore store real len of voices  - before view

        v0 = v0.view(-1)
        v1 = v1.view(-1)
        v2 = v2.view(-1)
        v3 = v3.view(-1)
 
        
        loss = self.loss_pitch(score_0, v0) + self.loss_pitch(score_1, v1) + self.loss_pitch(score_2, v2) + self.loss_pitch(score_3, v3)
        
        return loss

    def predict(self, sentences, sentences_len):

        # Compute the outputs from the linear units.
        scores_0, scores_1, scores_2, scores_3 = self.compute_outputs(sentences, sentences_len)

        # Select the top-scoring labels. The shape is now (max_len, n_sentences).
        predicted_0 = scores_0.argmax(dim=2)
        predicted_1 = scores_1.argmax(dim=2)
        predicted_2 = scores_2.argmax(dim=2)
        predicted_3 = scores_3.argmax(dim=2)       # prbl distr over 0-not there, 1-there   - argmax -> take higher probl of those 2
        
        ##  predicted_X should be enough for BS = 1;  
        return (
            [
                predicted_0[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
            [
                predicted_1[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
            [
                predicted_2[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
            [
                predicted_3[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
        )

